In [7]:
'''Project: “Potts Model Clustering”
Author : Lionel Yelibi, 2019, University of Cape Town.
Copyright SPC, 2019, 2020
Potts Model Clustering.
Agglomerative Fast Super-Paramagnetic Clustering
See pre-print: https://arxiv.org/abs/1908.00951
GNU GPL
This file is part of SPC
SPC is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.
SPC is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.
You should have received a copy of the GNU General Public License
along with this program.  If not, see <https://www.gnu.org/licenses/>.'''

import numpy as np
from sklearn.datasets import make_blobs
from numba import jit
import pandas as pd

In [23]:
''' the cluster function:
    compute the likelihood which occurs when two objects are clustered or
    the object own likelihood. By design if a cluster only containts one object
    its likelihood will be 0'''
@jit(nopython=True)
def clus_lc(gij,gii,gjj, ns=2):
    ''' variables description:
        ns is the size of the cluster.
        cs is the intracluster correlation.
        gij, gii, and gjj respectively are
        correlations relating to interactions between objects i and j, and
        their respective self-correlations. self-correlations (i.e. gii, gjj)
        are 1 for individual objects and >1 for clusters'''
        
    if ns==1:
        return 0
    ''' intracluster correlation'''   
    cs = 2*gij+gii+gjj
    ''' relatively low cs means noisy and suboptimal clusters.
    The coupling parameter gs (see paper) isn't not defined'''
    if cs<=ns:
        return 0
    return 0.5*( np.log(ns/cs) +  (ns - 1)*np.log( (ns**2 - ns) / ( ns**2 - cs) )  )

''' aspc only requires a correlation matrix as input:
    here we convert the correlation to a dictionary for convenience. adding new
    entries in a dict() is much faster than editing a numpy matrix'''

' aspc only requires a correlation matrix as input:\n    here we convert the correlation to a dictionary for convenience. adding new\n    entries in a dict() is much faster than editing a numpy matrix'

In [52]:
N = 100
data,_ = make_blobs(n_samples=N,n_features=5000,shuffle=False,centers=10,random_state=0)
'''  it is possible to provide a cluster number, if none is provide
set cn = None'''
cn = 5

#sol  = agglo_spc(np.corrcoef(data),cn=cn)

G = np.corrcoef(data)

In [53]:
GDF = pd.DataFrame(G)
GDF.to_csv("simuluatedData.csv")

In [54]:
N = len(G)
gdic = { i : { j : G[i,j] for j in range(N)} for i in range(N)}

    
''' tracker is dictionary which stores the objects member of the same clusters.
the data is stored as strings: i.e. cluster 1234 contains objects 210 & 890
which results in tracker['1234'] == '210_890' '''
tracker = { i:str(i) for i in range(N) }
    
''' the cluster size ns is stored in the ns_ array'''
ns_ = [1]*N
    
''' Create a list of object indices 'other_keys': at every iteration one object
is clustered and removed from the list. It is also removed if no suitable
optimal destination is found.'''
other_keys = list(range(N))

''' the operation stocks once there is only one object left to cluster as we
need two objects at the very least.'''

' the operation stocks once there is only one object left to cluster as we\nneed two objects at the very least.'

In [55]:
# tests
print(gdic)
print("----------------next object----------")
print(tracker)
print("----------------next object----------")
print(ns_)
print("----------------next object----------")
print(other_keys)

{0: {0: 1.0, 1: 0.9713516718094142, 2: 0.9708282498366982, 3: 0.9715279440940975, 4: 0.9714742132751208, 5: 0.9706976496690403, 6: 0.9710045912756334, 7: 0.9712565872815906, 8: 0.9716823711370118, 9: 0.9711249080768304, 10: -0.0006184653074024244, 11: 0.006140950581923421, 12: 0.001754619837629523, 13: -0.0008825661716263798, 14: 0.00255777447522317, 15: 0.0017552090921937942, 16: -0.001023305138932644, 17: 0.0021170325867597554, 18: 0.0005060793080946401, 19: -0.0014588710763701086, 20: 0.0011196711436786291, 21: 0.0009883640139257475, 22: 0.0023607804289029156, 23: 0.000202161133636326, 24: 0.007241332656956126, 25: 0.006293270706445185, 26: 0.005038982293130881, 27: 0.006222378359735894, 28: 0.003128360128765802, 29: 0.006295138910975891, 30: -0.0012802359050075682, 31: 0.004048999666796036, 32: -0.0025589393552827994, 33: -0.003060929229317411, 34: -0.0045524430147403155, 35: -0.0009563778422860215, 36: -0.003196513852489943, 37: 0.0007976790423752163, 38: -0.003555236950084917, 39

In [88]:
#WHILE LOOOOOOP

In [89]:
if len(other_keys)>1:
    node = np.random.choice(other_keys)
else:
    node = np.random.choice(list(tracker.keys()))
nbor = list(tracker.keys())
nbor.remove(node)
costs = np.zeros(len(nbor))
indices = [(node,key) for key in nbor]
node_lc = clus_lc(0,gdic[node][node],0,ns = ns_[node])
k=0
for i,j in indices:
    costs[k] = clus_lc(gdic[i][j],gdic[i][i],gdic[j][j],ns=ns_[i]+ns_[j]) - (node_lc+clus_lc(0,gdic[j][j],0,ns = ns_[j]))
    k+=1
            
''' find the optimal cost which will be the object clustered with node'''
next_merge = np.argmax(costs)

In [90]:
# tests
print(node)
print("----------------next object----------")
print(nbor)
print("----------------next object----------")
print(costs)
print("----------------next object----------")
print(indices)
print("----------------next object----------")
print(next_merge)

47
----------------next object----------
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 33, 34, 35, 38, 39, 40, 41, 42, 43, 44, 45, 46, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 78, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 101, 102]
----------------next object----------
[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  3.90125950e-05  1.65400960e-06
  2.63920699e-05  2.30978693e-05  8.98225674e-06  1.65713750e-05
  0.00000000e+00  3.38813459e-05  2.57558748e-06  1.81596627e-05
  1.96430990e-05  2.06890442e-06  3.71854509e-05  2.88545066e-05
  1.01539347e-05  1.49607125e-05  7.51579342e-06  6.48044270e-06
  2.07939993e-05  3.44934072e-06  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.0000

In [91]:
''' stopping conditions '''
if costs[next_merge]<=0:
    if len(other_keys)>1:
        ''' if no cost is positive then this node cannot be clustered further
        and must be removed from the list'''
        other_keys.remove(node)
        #continue
    elif not cn:
        ''' if no cluster number is provided then the routine has completed
        and tracker is the final solution'''
        cn = len(tracker)
        #continue
    elif cn:
        ''' if a cluster number is provided the routine continues and keeps
        merging'''
        #pass

In [92]:
costs[next_merge]

1.4438158115508473

In [93]:
''' on the other hand, the largest positive cost is the designated
object 'label_b' clustered to node which here is stored as 'label_a'.
new clusters 'new_label' take values superior to N.
tracker, as previously explained, stores joined strings of the clusters
contents'''
        
label_a = node
label_b = indices[next_merge][1]
new_label = len(ns_) # add 1 to the R implementation here...new label keeps overwritting the last object!!!

In [94]:
print(label_a)
print("----------------next object----------")
print(label_b)
print("----------------next object----------")
print(new_label)

47
----------------next object----------
41
----------------next object----------
103


In [95]:
''' removes merged elements and update others with the new cluster.
only do it when a positive cost is found.'''
if costs[next_merge]>0:
    other_keys = list(tracker.keys())
    other_keys.remove(label_a)
    other_keys.remove(label_b)
    other_keys.append(new_label)

In [96]:
''' Once a cluster is formed, the correlation matrix gdic and tracker need to
be updated with the new cluster and the cluster size must be updated with ns_'''
nbor.remove(label_b)
tracker[new_label]=tracker[label_a]+'_'+tracker[label_b]
gdic[new_label]={}
gdic[new_label][new_label] = 2*gdic[label_a][label_b] + gdic[label_a][label_a] + gdic[label_b][label_b]
ns_.append(ns_[label_a] + ns_[label_b])

In [97]:
tracker

{0: '0',
 1: '1',
 2: '2',
 3: '3',
 4: '4',
 5: '5',
 6: '6',
 7: '7',
 8: '8',
 9: '9',
 10: '10',
 11: '11',
 12: '12',
 13: '13',
 14: '14',
 15: '15',
 16: '16',
 17: '17',
 18: '18',
 19: '19',
 20: '20',
 21: '21',
 22: '22',
 23: '23',
 24: '24',
 25: '25',
 26: '26',
 27: '27',
 28: '28',
 29: '29',
 30: '30',
 31: '31',
 33: '33',
 34: '34',
 35: '35',
 38: '38',
 39: '39',
 40: '40',
 41: '41',
 42: '42',
 43: '43',
 44: '44',
 45: '45',
 46: '46',
 47: '47',
 48: '48',
 49: '49',
 50: '50',
 51: '51',
 52: '52',
 53: '53',
 54: '54',
 55: '55',
 56: '56',
 57: '57',
 58: '58',
 59: '59',
 60: '60',
 61: '61',
 62: '62',
 63: '63',
 64: '64',
 65: '65',
 66: '66',
 67: '67',
 68: '68',
 69: '69',
 70: '70',
 71: '71',
 72: '72',
 73: '73',
 74: '74',
 75: '75',
 76: '76',
 78: '78',
 80: '80',
 81: '81',
 82: '82',
 83: '83',
 84: '84',
 85: '85',
 86: '86',
 87: '87',
 88: '88',
 89: '89',
 90: '90',
 91: '91',
 92: '92',
 93: '93',
 94: '94',
 95: '95',
 96: '96',
 97: '97

In [98]:
for key in nbor:
    gdic[new_label][key] = gdic[label_a][key] + gdic[label_b][key]
    gdic[key][new_label] = gdic[label_a][key] + gdic[label_b][key]
    
del tracker[label_a]
del tracker[label_b]